# Metal Designz Scraping

## Imports and Setup

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv

# Set Variables
SLEEP_TIME = 5
STARTING_LINK = 'https://www.metaldesignz.com/jump-rings/'
LINK_FILE_PATH = '../../product_page_listings/metal_designz/metal_designz_products.csv'


## Get Data

### Find All Product Pages

Looking at their website it appears that the basic structure is a main page with side bar links to sub pages for each ring category. The sub pages link to product pages but if there are enough product pages they will be paginated.


In [2]:

# Helper function that gets the product links from a category page
def get_product_links(soup: BeautifulSoup, writer: csv.writer) -> None:
    product_grid = soup.find('ul', {'class': 'productGrid'})
    products = product_grid.find_all('li', {'class': 'product'})

    for product in products:
        link_tag = product.find('a', {'class': 'card-figure__link'})
        link = link_tag['href']
        writer.writerow([link])


# Get link to next page if applicable
def get_next_page(soup: BeautifulSoup) -> str:
    next_page = None
    pagination = soup.find('nav', {'class': 'pagination'})
    link_tag = pagination.find('a', {'aria-label': 'Next'})

    if link_tag:
        next_page = link_tag['href']
    
    return next_page


# From the starting link find all product pages
def page_parser(url: str, writer: csv.writer) -> None:

    # Get page and create a parsable object
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Get links to category pages
    sidebar = soup.find('aside', {'class': 'page-sidebar'})
    category_sidebar = sidebar.find(lambda tag: tag.name == 'div' and tag.find('h2', string='Jump Rings') is not None)
    category_links = set(map(lambda tag: tag['href'], category_sidebar.find_all('a')))

    # Iterate through top level links
    for num, link in enumerate(category_links):
        print(f"Checking cateogry: {num} -  {link}")
        sleep(5)

        # Get first page of category
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get the product links from the page
        get_product_links(soup, writer)

        # Get link to the next page
        next_page = get_next_page(soup)

        # Itterate through remaining pages if applicable
        while next_page:
            print(f'Found New Page: {next_page}')
            sleep(5)

            # Get new page            
            response = requests.get(next_page)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Print product links
            get_product_links(soup, writer)

            # Get next page if applicable
            next_page = get_next_page(soup)


# Use function to write links to file
with open(LINK_FILE_PATH, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    page_parser(STARTING_LINK, writer)


Checking cateogry: 0 -  https://www.metaldesignz.com/anodized-aluminum/
Found New Page: https://www.metaldesignz.com/anodized-aluminum/?page=2
Found New Page: https://www.metaldesignz.com/anodized-aluminum/?page=3
Checking cateogry: 1 -  https://www.metaldesignz.com/sterling-silver/
Checking cateogry: 2 -  https://www.metaldesignz.com/half-round-wire-rings/
Checking cateogry: 3 -  https://www.metaldesignz.com/new-anodized-aluminum/
Checking cateogry: 4 -  https://www.metaldesignz.com/gold-fill/
Found New Page: https://www.metaldesignz.com/gold-fill/?page=2
Found New Page: https://www.metaldesignz.com/gold-fill/?page=3
Checking cateogry: 5 -  https://www.metaldesignz.com/niobium/
Found New Page: https://www.metaldesignz.com/niobium/?page=2
Checking cateogry: 6 -  https://www.metaldesignz.com/copper-1/
Found New Page: https://www.metaldesignz.com/copper-1/?page=2
Found New Page: https://www.metaldesignz.com/copper-1/?page=3
Found New Page: https://www.metaldesignz.com/copper-1/?page=4
Fo

### Get Data From Product Pages